In [37]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [38]:
data = pd.read_csv('traindata.csv')
data.head(5)

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,tcp,http,SF,295,2584,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,101,255,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21,0
1,1,0,tcp,http,SF,312,839,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,223,255,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21,0
2,2,0,tcp,ftp,SF,1225,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,255,116,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11,1
3,3,0,tcp,http,SF,293,296,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.50,216,255,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21,0
4,4,0,tcp,X11,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.5,1.0,0.5,1.0,0.00,115,1,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11,0


In [39]:
data1 = pd.read_csv('testdata.csv')

In [40]:
all_services = np.concatenate([data['service'], data1['service']]).reshape(-1, 1)

In [41]:
enc = OrdinalEncoder()
data['protocol_type'] = enc.fit_transform(np.array(data['protocol_type']).reshape(-1, 1))
enc_f = OrdinalEncoder()
data['flag'] = enc_f.fit_transform(np.array(data['flag']).reshape(-1, 1))
enc_srv = OrdinalEncoder()
enc_srv.fit(all_services)
data['service'] = enc_srv.transform(np.array(data['service']).reshape(-1, 1))

In [42]:
# Top 5 selected using RFE + others
data = data.filter(['protocol_type', 'src_bytes', 'dst_bytes', 'dst_host_srv_count', 'count', 'level', 'service', 'flag', 'Class'], axis=1)
data.head(5)

,protocol_type,src_bytes,dst_bytes,dst_host_srv_count,count,level,service,flag,Class
0,1.0,295,2584,255,2,21,22.0,9.0,0
1,1.0,312,839,255,5,21,22.0,9.0,0
2,1.0,1225,2451,116,1,11,18.0,9.0,1
3,1.0,293,296,255,3,21,22.0,9.0,0
4,1.0,0,0,1,2,11,1.0,1.0,0


In [43]:
data1 = data1.filter(['protocol_type', 'src_bytes', 'dst_bytes', 'dst_host_srv_count', 'count', 'level', 'service', 'flag'], axis=1)
data1.head(5)

,protocol_type,src_bytes,dst_bytes,dst_host_srv_count,count,level,service,flag
0,tcp,30,93,232,1,16,pop_3,SF
1,tcp,0,0,5,240,21,private,S0
2,udp,52,54,255,511,17,private,SF
3,tcp,367,588,255,1,21,http,SF
4,icmp,1008,0,255,311,18,ecr_i,SF


In [44]:
data1['protocol_type'] = enc.transform(np.array(data1['protocol_type']).reshape(-1, 1))
data1['flag'] = enc_f.transform(np.array(data1['flag']).reshape(-1, 1))
data1['service'] = enc_srv.transform(np.array(data1['service']).reshape(-1, 1))

In [45]:
X = data.drop(['Class'], axis=1)
y = data['Class']

In [46]:
rs = RobustScaler()
X = rs.fit_transform(X)
data1 = rs.transform(data1)

In [48]:
clf = DecisionTreeClassifier(splitter='best', criterion='entropy', class_weight={0: 1, 1: 10}, 
                              max_depth=6, max_features=6, max_leaf_nodes=12, random_state=1218)
clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight={0: 1, 1: 10},
                       criterion='entropy', max_depth=6, max_features=6,
                       max_leaf_nodes=12, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=1218,
                       splitter='best')

In [49]:
# #recursive feature elimination
# from sklearn.feature_selection import RFE
# selector = RFE(clf, n_features_to_select=5, step=1)
# selector = selector.fit(X, y)
# selector.ranking_

Upon training the classifier on the initial feature set (all features), the above code snippet was run, and the top 5 selected features were as follows:
- protocol_type
- src_bytes
- dst_bytes
- dst_host_srv_count
- level


In [51]:
y_pred=clf.predict(data1)

In [52]:
id = np.arange(0, len(data1))
prediction = pd.DataFrame({'ID':id,'Class':y_pred},columns=['ID','Class'])
#prediction.head(2)
prediction.to_csv('prediction.csv',index=False)